In [1]:
import os,datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path
SEED_XML_DIR = '/data/darknet/python/xml/'
SEED_IMG_DIR = '/data/darknet/python/image/'
GENE_IMG_DIR = '../generated/image_expand_obj/'
GENE_XML_DIR = '../generated/xml_expand_obj/'
GENE_OBJ_DIR = '../generated/img_objects/'
SPECIAL_OGJ_DIR = '../manufactur/specail_obj/'